In [5]:
import numpy as np
import os 
import cv2 as cv
import dlib
from imutils.face_utils import shape_to_np
import csv
from math import atan, sqrt, floor
from tqdm import tqdm

In [7]:
from sklearn.preprocessing import LabelEncoder

In [17]:
class FeatureVector(object):
  def __init__(self, facialLandmarksDatFilePath, emotionLabels = ["angry", "disgust", "happy", "neutral", "sad", "surprise"]):
    self.detector = dlib.get_frontal_face_detector()
    self.predictor = dlib.shape_predictor(facialLandmarksDatFilePath)
    self.labelEncoder = LabelEncoder()
    self.labelEncoder.fit(sorted(emotionLabels))


  def __GetCenterOfGravityPoint(self, faceShape):
    # initialize to zero
    X_cog, Y_cog = (0, 0)

    # sum up the x and y coords of the points
    for i in range (len(faceShape)):
      X_cog += faceShape[i][0]
      Y_cog += faceShape[i][1]

    # divide by the total number of points
    X_cog = X_cog // len(faceShape)
    Y_cog = Y_cog // len(faceShape)

    return (X_cog, Y_cog)

  
  def __ComputeFeatureVector(self, faceShape, centerOfGravity):
    cogX, cogY = centerOfGravity
    # calculate the angle between the top point of nose and the tip of the nose for normalizing the image
    angle_nose = (faceShape[28][1] - faceShape[31][1]) / (faceShape[28][0] - faceShape[31][0])
    angle_nose = atan(angle_nose)

    feature_vector = []

    for i in range(len(faceShape)):
      X_relative = faceShape[i][0] - cogX
      Y_relative = faceShape[i][1] - cogY

      # euclidean distance
      euc = pow(X_relative, 2) + pow(Y_relative, 2)
      euc = sqrt(euc)
      
      if X_relative > 0:
        theta = atan(Y_relative / X_relative) - angle_nose
      else:
        theta = 0

      feature_vector.append(X_relative)
      feature_vector.append(Y_relative)
      feature_vector.append(euc)
      feature_vector.append(theta)
      
    return feature_vector


  def ConstructFeatures(self, imagePath):
    """
    Takes a single image, constructs the feature vector and returns it
    """
    img = cv.imread(imagePath)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    rects = self.detector(gray, 1)
    
    if (len(rects) > 0):
      rect = rects[0]
      faceShape = self.predictor(gray, rect)
      faceShape = shape_to_np(faceShape)

      centerOfGravityPoint = self.__GetCenterOfGravityPoint(faceShape)
      featureVector = self.__ComputeFeatureVector(faceShape, centerOfGravityPoint)

      label = imagePath.rsplit(os.sep, 2)[-2]
      featureVector.append(self.labelEncoder.transform([label])[0])

      return np.asarray(featureVector)

In [14]:
ckPreprocessedRoot = "/content/drive/My Drive/CSE445/CSE445 Dataset/ck-preprocessed"

In [16]:
fv = FeatureVector("/content/drive/My Drive/CSE445/shape_predictor_68_face_landmarks.dat")

In [19]:
def DumpFeatureVectorsToCsv(processedDataRoot, fileSavePath):
  featureVectors = []

  for folderPath, subdirs, files in os.walk(processedDataRoot):
    # skip first iteration
    if len(files) == 0:
      continue

    for fileName in tqdm(files):
      imagePath = os.path.join(folderPath, fileName)
      featureVector = fv.ConstructFeatures(imagePath)
      featureVectors.append(featureVector)
    
  featureVectors = np.asarray(featureVectors)
  np.savetxt(fileSavePath, featureVectors, delimiter = ",")

In [20]:
DumpFeatureVectorsToCsv(ckPreprocessedRoot, "/content/drive/My Drive/CSE445/CSE445 Dataset/ck_features.csv")

100%|██████████| 68/68 [01:21<00:00,  1.19s/it]


In [21]:
jaffePreprocessedRoot = "/content/drive/My Drive/CSE445/CSE445 Dataset/jaffe-preprocessed"
DumpFeatureVectorsToCsv(jaffePreprocessedRoot, "/content/drive/My Drive/CSE445/CSE445 Dataset/jaffe_features.csv")

100%|██████████| 30/30 [00:08<00:00,  3.46it/s]


In [22]:
kdefPreprocessedRoot = "/content/drive/My Drive/CSE445/CSE445 Dataset/kdef-preprocessed"
DumpFeatureVectorsToCsv(kdefPreprocessedRoot, "/content/drive/My Drive/CSE445/CSE445 Dataset/kdef_features.csv")

100%|██████████| 140/140 [00:36<00:00,  3.87it/s]
